In [ ]:
import os, glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
img_dir = "../input/petfinder-pawpularity-score/train/"
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
save_dir = "/kaggle/working/meta_images/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
def keep_aspect_resize(img, size, interpolation=cv2.INTER_CUBIC, cval=0):
    h, w = img.shape[:2]
    w = should_odd(w)
    h = should_odd(h)
    img = cv2.resize(img, (w, h), interpolation=interpolation)
    d = abs(w - h) // 2
    if w > h:
        img = cv2.copyMakeBorder(img, d, d, 0, 0, cv2.BORDER_CONSTANT, value=[cval]*3)
    elif h > w:
        img = cv2.copyMakeBorder(img, 0, 0, d, d, cv2.BORDER_CONSTANT, value=[cval]*3)
    else:
        pass
    
    h, w = img.shape[:2]
    assert w == h, "width {} not equal to height {}.".format(w, h)

    return cv2.resize(img, size, interpolation=interpolation)

def should_odd(n):
    if n % 2 != 0:
        n += 1
    return n

def aligm_meta(meta_df):
    meta_info = {} # {col: {text: , size: }}
    max_text_width = 0
    total_text_height = 0
    for col in meta_df.columns.tolist():
        if col == 'Id':
            continue
        val = str(meta_df[col].values[0])
        text = "{}:{}".format(col, val)
        meta_info[col] = {}
        meta_info[col]["text"] = text
        meta_info[col]["size"] = cv2.getTextSize(text, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.0, thickness=2)[0]
        w, h = meta_info[col]["size"]
        if w > max_text_width:
            max_text_width = w
        total_text_height += h + 10
            
    return meta_info, max_text_width, total_text_height
        
def draw_meta(img, meta_info, max_text_width, total_text_height):
    h, w = img.shape[:2]
    if h < total_text_height:
        img = img.reize(img, (total_text_height, total_text_height))
        h, w = img.shape[:2]
    img = cv2.copyMakeBorder(img, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value=[255]*3)
    img = cv2.copyMakeBorder(img, 0, 0, 0, max_text_width, cv2.BORDER_CONSTANT, value=[0]*3)
    start_h = 0
    start_w = w+1
    for col, meta in meta_info.items():
        text = meta["text"]
        t_w, t_h = meta["size"]
        start_h += t_h + 10
        cv2.putText(img,
                    text=text,
                    org=(start_w, start_h),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1.0,
                    color=(255,200,0),
                    thickness=2,
                    lineType=cv2.LINE_AA)
        
    return img

In [ ]:
for file in sorted(glob.glob(img_dir+"*")):
    name = file.split("/")[-1].split(".")[0]
    img = cv2.imread(file)
    save_path = save_dir + name + ".jpg"
    orig_h, orig_w = img.shape[:2]
    meta_df = train_df[train_df['Id'] == name]
    meta_df = meta_df.assign(Height=orig_h)
    meta_df = meta_df.assign(Width=orig_w)
    
    img = keep_aspect_resize(img, (512, 512))
    meta_info, max_text_width, total_text_height = aligm_meta(meta_df)
    img = draw_meta(img, meta_info, max_text_width, total_text_height)
    
    cv2.imwrite(save_path, img)
# print(meta_df)
# print(meta_info, max_text_width)
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.show()